In [ ]:
def compute_orders_pearls(self, product, order_depth):
    orders: list[Order] = []

    osell = collections.OrderedDict(sorted(order_depth.sell_orders.items()))
    obuy = collections.OrderedDict(sorted(order_depth.buy_orders.items(), reverse=True))

    sell_vol, best_sell_pr = self.values_extract(osell)
    buy_vol, best_buy_pr = self.values_extract(obuy, 1)

    cpos = self.position[product]

    mx_with_buy = -1

    for ask, vol in osell.items():
        if ((ask < 10000) or ((self.position[product]<0) and (ask == 10000))) and cpos < self.POSITION_LIMIT['PEARLS']:
            mx_with_buy = max(mx_with_buy, ask)
            order_for = min(-vol, self.POSITION_LIMIT['PEARLS'] - cpos)
            cpos += order_for
            assert(order_for >= 0)
            orders.append(Order(product, ask, order_for))

    mprice_actual = (best_sell_pr + best_buy_pr)/2
    mprice_ours = 10000

    undercut_buy = best_buy_pr + 1
    undercut_sell = best_sell_pr - 1

    bid_pr = min(undercut_buy, acc_bid-1) # we will shift this by 1 to beat this price
    sell_pr = max(undercut_sell, acc_ask+1)

    if (cpos < self.POSITION_LIMIT['PEARLS']) and (self.position[product] < 0):
        num = min(40, self.POSITION_LIMIT['PEARLS'] - cpos)
        orders.append(Order(product, min(undercut_buy + 1, acc_bid-1), num))
        cpos += num

    if (cpos < self.POSITION_LIMIT['PEARLS']) and (self.position[product] > 15):
        num = min(40, self.POSITION_LIMIT['PEARLS'] - cpos)
        orders.append(Order(product, min(undercut_buy - 1, acc_bid-1), num))
        cpos += num

    if cpos < self.POSITION_LIMIT['PEARLS']:
        num = min(40, self.POSITION_LIMIT['PEARLS'] - cpos)
        orders.append(Order(product, bid_pr, num))
        cpos += num
    
    cpos = self.position[product]

    for bid, vol in obuy.items():
        if ((bid > acc_ask) or ((self.position[product]>0) and (bid == acc_ask))) and cpos > -self.POSITION_LIMIT['PEARLS']:
            order_for = max(-vol, -self.POSITION_LIMIT['PEARLS']-cpos)
            # order_for is a negative number denoting how much we will sell
            cpos += order_for
            assert(order_for <= 0)
            orders.append(Order(product, bid, order_for))

    if (cpos > -self.POSITION_LIMIT['PEARLS']) and (self.position[product] > 0):
        num = max(-40, -self.POSITION_LIMIT['PEARLS']-cpos)
        orders.append(Order(product, max(undercut_sell-1, acc_ask+1), num))
        cpos += num

    if (cpos > -self.POSITION_LIMIT['PEARLS']) and (self.position[product] < -15):
        num = max(-40, -self.POSITION_LIMIT['PEARLS']-cpos)
        orders.append(Order(product, max(undercut_sell+1, acc_ask+1), num))
        cpos += num

    if cpos > -self.POSITION_LIMIT['PEARLS']:
        num = max(-40, -self.POSITION_LIMIT['PEARLS']-cpos)
        orders.append(Order(product, sell_pr, num))
        cpos += num

    return orders